## Note
Trying out different models and paramaters to see which performs the best

In [275]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [557]:
# Loading the evaluation data
dev = pd.read_csv("../csv_files/development.csv")

In [559]:
# Extracting the positions and removes the x and y column.
import numpy as np
pos_dev = dev[["x", "y"]]

## Dropping data from x and y 
dev = dev.drop(["x", "y"], axis=1)

In [566]:
## Removing pads with format: pads = [0, 7, 12, ..]
def drop_pads(input_list, df):
    for i in input_list:
        columns_to_remove = df.filter(like=f'[{i}]').columns
        df = df.drop(columns=columns_to_remove)
    return df

remove_pads = [0, 7, 12, 15, 16, 17]
dev_removed_noise = drop_pads(remove_pads, dev)

In [567]:
## Removing rms feature
def drop_rms_features(df):
    # Extract columns that start with 'rms'
    rms_columns = [col for col in df.columns if not col.startswith('rms')]

    # Create a new DataFrame without 'rms' columns
    df_without_rms = df[rms_columns] 
    return df_without_rms

dev_interesting_data = drop_rms_features(dev_removed_noise)

In [568]:
## Z-transformation of the data. Remember to scale accordingly to training data for eval data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(dev_interesting_data)
 
dev_interesting_data = pd.DataFrame(scaler.transform(dev_interesting_data), columns=dev_interesting_data.columns)

In [ ]:
# ## Reducing the dataset to X percent of original size to speed up model testing
# dev_interesting_data_sample = dev_interesting_data.sample(frac=0.25)
# pos_dev_sample = pos_dev.loc[dev_interesting_data.index]

In [569]:
## Splitting into train and validation set
X_train, X_val, pos_train, pos_val = train_test_split(dev_interesting_data, pos_dev, test_size=0.2, random_state=42)

## RandomForestRegressor

In [571]:
# RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
import math
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import GridSearchCV

numb_trees = 35
base_regressor = RandomForestRegressor(n_estimators=numb_trees, criterion="poisson", max_depth=30, max_features=0.3, bootstrap=True) 
mult_regr = MultiOutputRegressor(base_regressor)

mult_regr.fit(X_train, pos_train)

pos_pred = mult_regr.predict(X_val)

## Note on GridSearchCV
By splitting the problem into a regression problem for x-coordinate and y-coordinate I used GridSearchCV with these paramaters:
   
    param_grid = {  
        'max_depth': [20, 30, 40],  # Maximum depth of the tree
        'min_samples_split':[2, 4, 6], # Minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2, 4], # Minimum number of samples required to be at a leaf node
        'max_features': ["sqrt", "log2", None]
        }  
         
The best paramaters for x-coordinate and y-coordinate were: 
  
{'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}  
{'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 4}  

 
 This GridSearch is run on only 1% of the dataset in order to speed up the process.



In [575]:
# Metrics to evaluating model 
import sklearn.metrics as sm
import math
import numpy as np

def avg_euc_dist(pos_val, pos_pred):
    sum_square = 0
    for i in range(len(pos_val)):
        sum_square += math.sqrt((pos_val[i][0]-pos_pred[i][0])**2 + (pos_val[i][1]-pos_pred[i][1])**2)
    return sum_square/len(pos_val)    

def metrics_on_model(pos_val, pos_pred):
    print("Mean absolute error =", round(sm.mean_absolute_error(pos_val, pos_pred), 2)) 
    print("Mean squared error =", round(sm.mean_squared_error(pos_val, pos_pred), 2)) 
    print("Median absolute error =", round(sm.median_absolute_error(pos_val, pos_pred), 2)) 
    print("Explain variance score =", round(sm.explained_variance_score(pos_val, pos_pred), 2)) 
    print("R2 score =", round(sm.r2_score(pos_val, pos_pred), 2))
    print("Mean eucledian distance =", round(avg_euc_dist(pos_val, pos_pred), 2))

metrics_on_model(pos_val.to_numpy(), pos_pred)

Mean absolute error = 2.85
Mean squared error = 14.59
Median absolute error = 2.29
Explain variance score = 1.0
R2 score = 1.0
Mean eucledian distance = 4.5


## Results:

Test 1:  
    Data: reomving pads (0, 7, 12, 15, 16, 17)  
    Number of trees: 10   
    Mean euc dist: 5.77  

Test 2:  
    Data: reomving pads (0, 7, 12, 15, 16, 17) and removing rms feature  
    Number of trees: 10  
    Mean euc dist: 5.69

Test 3:    
    Data: Sampeled 25% of the data to speed up model training. Reomving pads (0, 7, 12, 15, 16, 17) and removing rms feature.  
    Number of trees: 10  
    Mean euc dist: 6.56  

Test 4:    
    Data: Sampeled 25% of the data to speed up model training. Reomving pads (0, 7, 12, 15, 16, 17) and removing rms feature.  
    Number of trees: 30  
    Mean euc dist: 6.16  

Test 4:    
    Data: With PCA. Sampeled 25% of the data to speed up model training. Reomving pads (0, 7, 12, 15, 16, 17) and removing rms feature.  
    Number of trees: 30  
    Mean euc dist: 10.29

Test 5:        
    Data: Reomving pads (0, 7, 12, 15, 16, 17) and removing rms feature. Added RobustScaling.   
    Number of trees: 50  
    Mean euc dist: 5.31  

Test 6:  
    Data: Reomving pads (0, 7, 12, 15, 16, 17) and removing rms feature. Added column: pmax[5]*area[5] 
    Number of trees: 30  
    Mean euc dist: 6.11

Test 7:   
    Data: Reomving pads (0, 7, 12, 15, 16, 17) and removing rms feature.  
    Number of trees: 25  
    Mean euc dist: 6.58  

 

## Using the multiple_reg_model on the evaluation set

In [576]:
ev_data = pd.read_csv("../csv_files/evaluation.csv")

In [577]:
# Extracting the ID
eval_id = ev_data["Id"]

# Dropping the Id column from the ev_data
ev_data = ev_data.drop(["Id"], axis=1)

In [578]:
# Formatting the position array to a string to be used in the .csv file 
def pred_to_string(prediction_array):
    pred_column = []
    for i in range(len(prediction_array)):
        pos_string = (str(prediction_array[i][0]) + "|" + str(prediction_array[i][1]))
        pred_column.append(pos_string)
    return pred_column
        

In [579]:
## Preprocessing 
remove_pads = [0, 7, 12, 15, 16, 17]
ev_data = drop_pads(remove_pads, ev_data) # Remove pads
ev_data = drop_rms_features(ev_data) # Remove rms feature 
ev_data = pd.DataFrame(scaler.transform(ev_data), columns=ev_data.columns) # Z-transform with mean and std from training data

In [580]:
# Predicting the evaluation results
mult_regr_eval = mult_regr.predict(ev_data)
pos_pred = pred_to_string(mult_regr_eval) # Formatting the predictions 

In [581]:
# Creating a df and .csv file to be submitted. Saved in submission_file folder
mult_reg_submission = pd.DataFrame({'Id': eval_id, 'Predicted': pos_pred})
mult_reg_submission.to_csv("../DataScienceLab_Project/submission_files/mult_reg_rand_forest_tuned_hyperparam_z_score_correct_pads.csv", index=False)